# Secp256k1

Constants from https://en.bitcoin.it/wiki/Secp256k1

In [1]:
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
F = FiniteField(p)

In [2]:
a = 0
b = 7
E = EllipticCurve(F, [a, b])

In [3]:
G_x = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
h = 1 # cofactor
G = B = E.lift_x(G_x)

In [4]:
assert(G.order() == 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141)

In [5]:
assert(E.order() / G.order() == h)

# Key generation

In [6]:
N = FiniteField(G.order())

In [7]:
def keypair (x = None):
    if x is None:
        x = int(N.random_element(1, N.order() - 1))
    P = x * G
    return (x, P)

# ECDSA

In [8]:
import hashlib
def sign (m, d):
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    k = int(N.random_element(1, N.order() - 1))
    R = k * G # (k, r) is like a one-time key pair
    r = int(R.xy()[0]) % N.order()
    assert(r % N.order() != 0)
    s = (e + d*r)/k % N.order()
    return [r,s]
    

In [9]:
def verify (sig, m, P):
    r, s = sig
    assert(0 < r < N.order())
    assert(0 < s < N.order())
    e = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    u1 = e / s % N.order()
    u2 = r / s % N.order()
    R = u1 * G + u2 * P
    v = int(R.xy()[0]) % N.order()
    return v == r

In [212]:
def sign2contract (m, d, c):
    hashedMessage = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    k = int(N.random_element(1, N.order() - 1))
    R = k * G # (k, r) is like a one-time key pair
    r = int(R.xy()[0]) % N.order()
    e = (k + int(hashlib.sha1(str(r) + c).hexdigest(), 16)) % N.order()
    Q = e * G
    q = int(Q.xy()[0]) % N.order()
    assert(r % N.order() != 0)
    s = (hashedMessage + d*q)/e % N.order()
    return [q, s], R

In [213]:
sk, pk = keypair()

In [214]:
commitSig = sign2contract('aaa', sk, 'bbb')

In [215]:
commitSig

([32246202784481761909328952743724046103440961076730565433897776360619657156729,
  10379687942096862383487235196010857466036345146583834602866457460329093864092],
 (56306156639224588148528766558006526624175635790609727167523915500083737435713 : 92798650632258975534137579122169765570429511544839700677349880248910649404454 : 1))

In [224]:
def verify2contract (m, pk, c, commit2sig):
    [r, s], K  = commit2sig
    assert(0 < r < N.order())
    assert(0 < s < N.order())
    hashedMessage = int(hashlib.sha1(m).hexdigest(), 16) % N.order()
    u1 =  hashedMessage / s % N.order()
    u2 = r / s % N.order()
    R = u1 * G + u2 * pk
    v = int(R.xy()[0]) % N.order()
    w = int(K.xy()[0]) % N.order()
    T = K + int(hashlib.sha1(str(w) + c).hexdigest(), 16) * G
    t = int(T.xy()[0]) % N.order()
    return v == r & r == t

In [225]:
verify2contract ('aaa', pk, 'bbb', commitSig)

True